In [33]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
# Load the image
inputfile="img2597810ANPR_L00.jpg"
outputfile="out.jpg"
path1 = "/home/user/Downloads/DAY2/SET7A1/"
img = cv2.imread(path1+inputfile)

background= cv2.imread("back.jpg")

per=50
width= int( (img.shape[1]*per) / 100)
height= int ((img.shape[0]*per) /100 )
dim=(width, height)
img = cv2.resize(img,dim, interpolation = cv2.INTER_AREA)
image=img
per1=50
width1= int( (background.shape[1]*per) / 100)
height1= int ((background.shape[0]*per) /100 )
dim1=(width1, height1)
background = cv2.resize(background,dim1, interpolation = cv2.INTER_AREA)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = np.array(255*(img/255)**2,dtype='uint8')
image=img

background = cv2.cvtColor(background, cv2.COLOR_BGR2RGB)
img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
background= cv2.cvtColor(background, cv2.COLOR_RGB2HSV)
H, S, V = cv2.split(img)
BH, BS, BV= cv2.split(background)
alpha1 = 0.2 ; beta1 = 0.8 ; thetas= 0.8; thetah= 0.8
s=V
import sys
V = np.float32(V)/255
BV= np.float32(BV)/255
S = np.float32(S)/255
BS= np.float32(BS)/255
H = np.float32(H)/255
BH= np.float32(BH)/255
s= V.copy()

for i in range(H.shape[0]):
    for j in range(H.shape[1]):
        try:
            x= V[i,j]/ BV[i,j]
        except ZeroDivisionError:
            x = 0
        
        if ( (alpha1 <= x <= beta1) and (abs(S[i,j]-BS[i,j])<= thetas) and (abs(H[i,j]-BH[i,j])<=thetah)) :
            s[i,j]=1
        else:
            s[i,j]=0
            
s=s*255

s = np.array(s,dtype=np.uint8)
median = cv2.medianBlur(s, 5)
#median =s
kernel = np.ones((5, 5), np.uint8)
closing = cv2.morphologyEx(median, cv2.MORPH_CLOSE, kernel)
opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)

#opening = s
opening = np.array(opening,dtype=np.uint8)

result = image.copy()
height=result.shape[0]
width=result.shape[1]
blank_image = np.zeros((height,width,3), np.uint8)
blank_image[opening!=0] = image[opening!=0]

blank_image1=np.uint8(np.double(blank_image)*1.2 + 5)


claheimg = np.zeros((height,width,3), np.uint8)
claheimg[opening==0]=image[opening==0]
clahe = cv2.createCLAHE(clipLimit=9.0,tileGridSize=(5, 5))

claheimg[:,:,0] = clahe.apply(blank_image1[:,:,0])
claheimg[:,:,1] = clahe.apply(blank_image1[:,:,1])
claheimg[:,:,2] = clahe.apply(blank_image1[:,:,2])


claheimg[opening==0]=image[opening==0]



#claheimg = np.array(255*(claheimg/255)**0.8,dtype='uint8')
# claheimg[:,:,0] = clahe.apply(claheimg[:,:,0])
# claheimg[:,:,1] = clahe.apply(claheimg[:,:,1])
# claheimg[:,:,2] = clahe.apply(claheimg[:,:,2])

cv2.imwrite(path1+outputfile,claheimg)

True

In [9]:
claheimg = np.array(claheimg,dtype=np.uint8)
rgb_planes = cv2.split(claheimg)

result_planes = []
result_norm_planes = []
arr = np.array([])
for plane in rgb_planes:

    dilated_img = cv2.dilate(plane, kernel, iterations = 5)
    dilated_img = np.array(dilated_img,dtype=np.uint8)
    #bg_img = cv2.medianBlur(dilated_img, 21)
   #bg_img  = np.array(bg_img ,dtype=np.uint8)
    bilateral = cv2.bilateralFilter(dilated_img, 9, 25, 25)
    bilateral = np.array(bilateral,dtype=np.uint8)
    diff_img = 255- cv2.absdiff(plane, bilateral)
    #diff_img = 255- cv2.absdiff(plane, bg_img)
    norm_img = cv2.normalize(diff_img, arr, alpha=0, beta=  255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    result_planes.append(diff_img)
    result_norm_planes.append(norm_img)

result = cv2.merge(result_planes)
result_norm = cv2.merge(result_norm_planes)
outputfile1="outnorm.jpg"

In [4]:
cv2.imwrite(path1+outputfile1,result_norm)

True